In [ ]:
`import requests
import json
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from google.colab import userdata

In [ ]:
GB_API = 'https://www.geoboundaries.org/api/current/gbOpen/HTI/ADM1/'

In [ ]:
r =  requests.get(GB_API)
data = r.json()
type(data)

dict

In [ ]:
data.keys()

dict_keys(['boundaryID', 'boundaryName', 'boundaryISO', 'boundaryYearRepresented', 'boundaryType', 'boundaryCanonical', 'boundarySource', 'boundaryLicense', 'licenseDetail', 'licenseSource', 'boundarySourceURL', 'sourceDataUpdateDate', 'buildDate', 'Continent', 'UNSDG-region', 'UNSDG-subregion', 'worldBankIncomeGroup', 'admUnitCount', 'meanVertices', 'minVertices', 'maxVertices', 'meanPerimeterLengthKM', 'minPerimeterLengthKM', 'maxPerimeterLengthKM', 'meanAreaSqKM', 'minAreaSqKM', 'maxAreaSqKM', 'staticDownloadLink', 'gjDownloadURL', 'tjDownloadURL', 'imagePreview', 'simplifiedGeometryGeoJSON'])

In [ ]:
nr = requests.get(data['gjDownloadURL'])
admin1 = nr.json()
type(admin1)

dict

In [ ]:
i = 0
for county in admin1['features']:
    # do something with the state
    prop = county['properties']
    print(prop['shapeName'])

    i +=1


print(i)

Agat
Barrigada
Hagåtña
Mongmong-Toto-Maite
Sinajana
Talofofo
Agana Heights
Santa Rita
Asan
Dededo
Inarajan
Merizo
Tamuning
Chalan Pago-Ordot
Mangilao
Umatac
Piti
Yigo
Yona
19


In [ ]:
def get_geoboundaries(iso,admin_level=1):
    '''Fetch the geoboundaries admin data for the supplied country
    based on the three-letter ISO. Returns a geojson file. '''
    parameters = {
        'ISO': iso,
        'ADM': f'ADM{admin_level}'
    }
    r = requests.get(GB_API, params=parameters)
    dlpath = r.json()[0]['gjDownloadURL']
    return requests.get(dlpath).json()

NameError: name 'USA' is not defined

In [ ]:
countries = pd.read_csv('/content/drive/MyDrive/Geoboundaries gemini/country_iso_codes.csv')
countries.head()

,country_name,iso_2,iso_3
0,Afghanistan,AF,AFG
1,Åland Islands,AX,ALA
2,Albania,AL,ALB
3,Algeria,DZ,DZA
4,American Samoa,AS,ASM


In [ ]:
!pip install -q -U google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 4.2 MB/s eta 0:00:00
ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 441, in run
    conflicts = self._dete

In [ ]:
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
from google.colab import userdata



In [ ]:
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)



In [ ]:
model = genai.GenerativeModel('gemini-pro')

In [1]:
import requests
import pandas as pd
import time
import os

output_folder = "/content/drive/MyDrive/Gemini Countries ADMIN 1"

iso_codes = ["VNM", "VUT", "WLF", "WSM", "YEM", "ZAF", "ZMB", "ZWE"]

def save_progress(iso_code, region_name):
    with open("progress.txt", "w") as f:
        f.write(f"{iso_code},{region_name}")

def load_progress():
    if os.path.exists("progress.txt"):
        with open("progress.txt", "r") as f:
            content = f.read().strip()
            if content:
                iso_code, region_name = content.split(",")
                return iso_code, region_name
    return None, None

def clear_progress():
    if os.path.exists("progress.txt"):
        os.remove("progress.txt")

# Load progress
last_iso_code, last_region_name = load_progress()

# Iterate over the ISO codes
iso_code_started = not bool(last_iso_code)  # Start if no progress is saved

for iso_code in iso_codes:
    if iso_code_started or iso_code == last_iso_code:
        iso_code_started = True
        GB_API = f'https://www.geoboundaries.org/api/current/gbOpen/{iso_code}/ADM1/'

        try:
            r = requests.get(GB_API)
            r.raise_for_status()
            data = r.json()
        except requests.exceptions.HTTPError as e:
            if r.status_code == 404:
                print(f"Error fetching data for {iso_code}: {e}. Skipping to next ISO code.")
                continue  # Skip to the next ISO code
            else:
                print(f"Error fetching data for {iso_code}: {e}. Retrying...")
                time.sleep(5)
                continue  # Retry the same ISO code

        while True:
            try:
                nr = requests.get(data['gjDownloadURL'])
                nr.raise_for_status()
                admin2 = nr.json()
                break
            except requests.exceptions.RequestException as e:
                print(f"Error downloading data for {iso_code}: {e}. Retrying...")
                time.sleep(5)

        gemini_results = {}

        print(f"ISO Code: {iso_code}")

        region_started = not bool(last_region_name)  # Start if no region progress is saved

        for region in admin2['features']:
            prop = region['properties']
            region_name = prop['shapeName']

            if region_started or region_name == last_region_name:
                region_started = True

                print(region_name, iso_code)

                while True:
                    try:
                        time.sleep(1)
                        gemini_response = model.generate_content(
                            f"GDP breakdown of {region_name}, {iso_code} by sectoral contributions from agriculture, industry, and services, no descriptions, percentages.")
                        formatted_gemini_response = gemini_response.text
                        gemini_results[region_name] = formatted_gemini_response

                        save_progress(iso_code, region_name)  # Save progress
                        break
                    except Exception as e:
                        print(f"Error processing region {region_name}: {e}. Retrying...")
                        time.sleep(5)

        for region_name, gemini_result in gemini_results.items():
            df = pd.DataFrame({"GDP Breakdown": [gemini_result]})
            file_path = f"{output_folder}/{iso_code}_{region_name}.xlsx"
            df.to_excel(file_path, index=False)

        print(f"Total regions processed for {iso_code}: {len(gemini_results)}")

clear_progress()  # Clear progress after successful completion



ISO Code: VNM
An Giang VNM
Error processing region An Giang: name 'model' is not defined. Retrying...


KeyboardInterrupt: 